# Configurações

![alt text](https://github.com/elthonf/plataformas-cognitivas-docker/blob/master/images/capa.png?raw=1)

In [ ]:
#@title Parâmetros da solução
#@markdown Informe a quantidade de clientes que devem ser analisados.
qtde_clientes = 6  #@param {type: "slider", min: 1, max: 10}

#@markdown Informe o IP do servidor de predição da sua solução:
my_server_ip = "34.73.51.91" #@param {type:"string"}

#@markdown Informe a porta do servidor de predição da sua solução, se nada foi alterado, deve ser 443:
my_server_port = "443" #@param {type:"string"}


#@markdown Ah, e não se esqueça de executar esta célula!


In [ ]:
import requests
import pandas as pd
import json

# Obtenção dos empréstimos pendentes de análise

Vamos agora obter os clientes que estã precisand de empréstimo e exibí-los na tela

In [ ]:
url = "https://us-central1-emf-teacher.cloudfunctions.net/function-aulas-getclient?qtde={0}".format(qtde_clientes)
headers = {'Content-Type': 'application/json'}
response = requests.request("GET", url, headers=headers)
response_content = response.text.encode('utf8').decode()
clientes = pd.read_json(response_content)

In [ ]:
clientes

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade
30834,"Saabira, el-Syed",4520.123105,56.394563,1,1,1,1,0,2
38320,"Rashida, Nguyen",4764.292325,47.615893,0,1,1,0,1,2
49541,"Dae Von, Cooley",4037.474433,41.216345,0,0,1,0,1,2
19032,"Kendell, Wolfe",5127.496647,39.389761,0,0,1,0,0,0
234,"Brittany, Arou",2681.429172,43.519453,0,1,0,0,0,1
23578,"Trieu, Moore",3434.354938,57.823141,1,0,0,0,1,2


# Predição

O cliente agora será submetido a dois modelos de Machine Learning preparados para a análise.

In [ ]:
url = "http://{}:{}/predict".format(my_server_ip, my_server_port)
headers = {'Content-Type': 'application/json'}
conteudo = clientes.to_json()

In [ ]:
response01 = requests.request("POST", url + "?model=modelo01", headers=headers, data=conteudo)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

{'prediction': [0, 0, 0, 0, 1, 1],
 'proba': [[0.9960620393366382, 0.00393796066336179],
  [0.9991321277113556, 0.0008678722886443633],
  [0.977846399853593, 0.022153600146406993],
  [0.9988999599313592, 0.0011000400686407912],
  [0.0036340050349890074, 0.996365994965011],
  [0.4803659803046114, 0.5196340196953886]]}

In [ ]:
df_01 = pd.Series(respostas01['prediction'], index = clientes.index, name="PredicaoML01")
df_01B = pd.Series(respostas01['proba'], index = clientes.index, name="PredicaoML01B")

In [ ]:
response02 = requests.request("POST", url + "?model=modelo02", headers=headers, data=conteudo)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'prediction': [0.015833372701813543,
  0.015338556935420201,
  0.036763734206683406,
  1.052660699597227,
  0.5455241547438923,
  1.0141180177491107]}

In [ ]:
df_02 = pd.Series(respostas02['prediction'], index = clientes.index, name="PredicaoML02")

In [ ]:
pd.DataFrame(clientes.loc[:,"nome"])\
    .merge(right= df_01, left_index=True, right_index=True)\
    .merge(df_01B, left_index=True, right_index=True)\
    .merge(df_02, left_index=True, right_index=True)

,nome,PredicaoML01,PredicaoML01B,PredicaoML02
6974,"Sirajuddeen, al-Azad",0,"[0.9791184653830104, 0.020881534616989557]",0.015833
40326,"Grant, Reddy",0,"[0.9349983259581628, 0.06500167404183724]",0.015339
9441,"Efrain, Huerta",0,"[0.9092766938346273, 0.09072330616537275]",0.036764
6676,"Sayra, Espinoza III",1,"[0.01312665515746747, 0.9868733448425325]",1.052661
41733,"Marco, Amon",1,"[0.17772761648070168, 0.8222723835192983]",0.545524
8123,"Noemi, Trevizo",1,"[0.000673604386141835, 0.9993263956138582]",1.014118


### Conclusão:

Devemos **conceder empréstimo** solicitado pelos 3 clientes que possuem menor probabilidade de default (não honrar a dívida), são eles:
- Sirajuddeen, al-Azad	(probab. de default 1.58%)
- Grant, Reddy (probab. de default 1.53%)
- Efrain, Huerta (probab. de default 3.76)

Podemos dar um empréstimo com uma alta taxa de juros para o cliente **Marco, Amon**, já que sua probabilidade de default é de 54.55% <br>

Os clientes **Sayra, Espinoza III** e **Noemi, Trevizo** tem possibilidade de default altíssima (pelo modelo é superior a 100%, ou seja é certo que não honrarão a dívida mesmo cobrando uma taxa de juros mais alta), para estes dois clientes devemos **negar o empréstimo**.